---

# Очистка данных

***Мусор на входе модели $\to$ мусор на выходе.***

Ни одна модель машинного обучения не выдаст осмысленных результатов, если вы предоставите ей сырые данные. После формирования выборки данных их необходимо очистить.

In [102]:
#main
import pandas as pd
import numpy as np

#plot
from matplotlib import pyplot as plt
import seaborn as sns

#
from tqdm import tqdm 
import re

pd.set_option("display.max_columns", 300)


# Имеем :
PATH_TRAIN = './data/train_sber.csv'
PATH_TEST = './data/test.csv'
PATH_MACRO = './data/macro.csv'

**У нас есть достаточно большой DataSet с кучей признаков много пропусков и мусорных значений. Будем обрабатывать!**

In [2]:
df_train = pd.read_csv(PATH_TRAIN)
df_makro = pd.read_csv(PATH_MACRO)
df_train_y = df_train['price_doc']
df_train.drop(columns='price_doc', axis=1, inplace=True)

**Создадим функцию для сестиматизации работы с признаками**

In [40]:
def moustache(s):
    # Принемает series
    # Возращает значения усов нормального распределения
    q75,q25 = np.percentile(s,[75,25])
    iqr = q75 - q25
    return q25 - 1.5*iqr, q75 + 1.5*iqr

def data_cleanup_table(df):
    # Возвращает таблицу для EDA с анализом признаков
    
    # Создание таблицы
    columns = ['Признак', 'Тип_данных', 'Пропущенных_значений', 
           'Выбросов', 'Ошибочных_значений', 'Управление', 'Комментарий']
    table = pd.DataFrame(columns=columns)
    table['Признак'] = df.columns
    df_len = len(df)
    print('Создание таблицы...')
    #
    for sign in df.columns:
        table.loc[table['Признак'] == sign, 'Тип_данных'] = df[sign].dtype
    
    
    # Анализ пропущенных значений
    for sign in tqdm(df.columns, desc='Анализ пропущенных значений\t'):
        table.loc[table['Признак'] == sign, 'Пропущенных_значений'] = df[sign].isna().sum()
        if df[sign].isna().sum() / df_len > 0.2:
            table.loc[table['Признак'] == sign, 'Управление'] = 'Удалить'
            table.loc[table['Признак'] == sign, 'Комментарий'] = 'Пропусков больше 20% '
            
    
    # Анализ выбрасов
    for sign in tqdm(df.columns, desc='Анализ выбросов\t\t\t'):
        if df[sign].dtype == 'object':
            table.loc[table['Признак'] == sign, 'Выбросов'] = -1
        else:
            tmp = moustache(df[sign]) # возращает границы усов (а, б)
            table.loc[table['Признак'] == sign, 'Выбросов'] = df[(df[sign] < tmp[0]) |
                                                                            (df[sign] > tmp[1])][sign].count()      
    
    return table

In [41]:
table = data_cleanup_table(df_train)
table.head()

Создание таблицы...


Анализ выбросов			: 100%|██████████| 291/291 [00:02<00:00, 106.60it/s]


,Признак,Тип_данных,Пропущенных_значений,Выбросов,Ошибочных_значений,Управление,Комментарий
0,id,int64,0,0,NaN,NaN,NaN
1,timestamp,object,0,-1,NaN,NaN,NaN
2,full_sq,int64,0,963,NaN,NaN,NaN
3,life_sq,float64,6383,0,NaN,Удалить,Пропусков больше 20%
4,floor,float64,167,0,NaN,NaN,NaN


**Для заполнения ошибочных значений надо вникнуть в суть признаков. В папке data/ файл data_dictionary.txt содержит описание всех признаков**

id, timestamp, full_sq, life_sq, floor, max_floor, material, build_yea, num_room, kitch_sq, state, product_type, sub_area

*В ручную просмотрев основные признаки найдем ошибочные значения*

---

**Отберем не адекватные признаки записав их в переменную false_col**

product_type: owner-occupier purchase or investment - Вряд ли имеет значения для стоимости жилья  
Признак (full_all: subarea population) полностью заменяет список признаков : 'male_f', 'female_f', 'young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female' и других по шаблону n_m_{all|male|female}: population between n and m years old  
build_count_*: buildings in the subarea by construction type or year  - Не нужно


In [122]:
false_col = ['product_type', 'male_f', 'female_f']

In [123]:
# Для получения списка признаков описанных в data_dictionary.txt с помощь "_*"

false_col += [x for x in df_train.columns if (re.match(r'young_', x) != None) |
 (re.match(r'work_', x) != None) | (re.match(r'ekder_', x) != None)]
false_col += [x for x in df_train.columns if (re.match(r'\d+_\d+_', x) != None)]
false_col += [x for x in df_train.columns if (re.match(r'build_count_', x) != None)]

**Удалить после решения**

Признаки к которым можно присмотреться и проанализировать

x_count_500: the number of x within 500m of the property  
x_part_500: the share of x within 500m of the property  
_sqm_: square meters  
 
[x for x in df_train.columns if (re.search(r'_count_500', x) != None)]  
[x for x in df_train.columns if (re.search(r'_part_500', x) != None)]

In [69]:
# Любая площадь, большая всей не жилой площади явно ошибка
def sq_valid(df, table):
    table.loc[table['Признак'] == 'life_sq', 'Ошибочных_значений'] = df[df['full_sq'] < df['life_sq']]['id'].count()
    table.loc[table['Признак'] == 'kitch_sq', 'Ошибочных_значений'] = df[df['full_sq'] < df['kitch_sq']]['id'].count()
    
def false_values(df, table):
    sq_valid(df, table)

false_values(df_train, table)

In [117]:
print(re.match(r'\d+_\d+_', '10324_10_qwedqwe'))

<re.Match object; span=(0, 9), match='10324_10_'>


In [80]:
df_train_y

0         5850000
1         6000000
2         5700000
3        13100000
4        16331452
           ...   
30466     7400000
30467    25000000
30468     6970959
30469    13500000
30470     5600000
Name: price_doc, Length: 30471, dtype: int64

In [92]:
np.correlate(df_train['full_all'], df_train['male_f'])

array([1415683265000626], dtype=int64)

In [83]:
np.correlate([1,2], [1,2])

0         86206
1         76284
2        101982
3         21155
4         28179
          ...  
30466     61396
30467    116742
30468     17790
30469     83844
30470     72131
Name: full_all, Length: 30471, dtype: int64